In [27]:
from run import *
from utils import *

device = "cuda" if torch.cuda.is_available() else "cpu"

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
run_name = "7sd8l0oc"
root_dir = os.path.join("..", "results", "DPT_2", run_name)
checkpoint_file = get_best_checkpoint(root_dir, 'epoch')

model = DPTSolver.load_from_checkpoint(checkpoint_file).cpu()#.to(device)

In [236]:
config = model.config
config['batch_size'] = 1

dl = get_dataloaders(config)

train_offline_dataloader = dl["train_dataloaders"]
val_offline_dataloader = dl["val_dataloaders"][0]
val_online_dataloader = dl["val_dataloaders"][1]

train_offline_dataset = train_offline_dataloader.dataset
val_offline_dataset = val_offline_dataloader.dataset
val_online_dataset = val_online_dataloader.dataset

train_offline_batch = next(iter(train_offline_dataloader))
val_offline_batch = next(iter(val_offline_dataloader))
val_online_batch = next(iter(val_online_dataloader))

train_offline_sample = train_offline_dataset[0]
val_offline_sample = val_offline_dataset[0]
val_online_sample = val_online_dataset[0]

train_offline_dataset: 4000
val_offline_dataset: 1000
val_online_dataset: 1000


In [238]:
outputs = model.model(
    x=sample["x"].unsqueeze(0), 
    y=sample["y"].unsqueeze(0)
)
predictions = model.get_predictions(outputs, sample["x_min"].unsqueeze(0))

RuntimeError: prob_dist must be 1 or 2 dim

In [237]:
# train_offline_batch = next(iter(train_offline_dataloader))
outputs = model._offline_step(train_offline_sample)
metrics = model.get_loss(**outputs) | model.get_metrics(**outputs)

print('predictions', outputs['predictions'].shape, outputs['predictions'].dtype)
print('targets', outputs['targets'].shape, outputs['targets'].dtype)
for key, val in metrics.items():
    print(f'{key}: {val.item():.5f}')

RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
val_offline_batch = next(iter(val_offline_dataloader))
outputs = model._offline_step(val_offline_batch)
metrics = model.get_loss(**outputs) | model.get_metrics(**outputs)

print('predictions', outputs['predictions'].shape, outputs['predictions'].dtype)
print('targets', outputs['targets'].shape, outputs['targets'].dtype)
for key, val in metrics.items():
    print(f'{key}: {val.item():.5f}')

predictions torch.Size([1, 4, 1]) torch.float32
targets torch.Size([1, 1]) torch.float32
loss: 0.00210
accuracy: 0.00000
mae: 0.04579


In [233]:
val_online_batch = next(iter(val_online_dataloader))
outputs = model._online_step(val_online_batch)
results = model.get_loss(**outputs) | model.get_metrics(**outputs)

print('predictions', outputs['predictions'].shape, outputs['predictions'].dtype)
print('targets', outputs['targets'].shape, outputs['targets'].dtype)
for key, val in metrics.items():
    print(f'{key}: {val.item():.5f}')

predictions torch.Size([1, 4, 1]) torch.float32
targets torch.Size([1, 1]) torch.float32
loss: 0.00141
accuracy: 0.00000
mae: 0.03761


In [235]:
transform = lambda x: ''.join([f'{x:.6f}' for x in x.tolist()])

for i, x in enumerate(outputs["predictions"][0]):
    y = val_online_batch["problem"][0].target(x.detach().numpy())
    print(f'observed: {i} suggested: x = {transform(x)}, y = {y:.6f}')
print(f'groun truth: x = {transform(sample["problem"].info["x_min"])}, y = {sample["problem"].info["y_min"]:.6f}')

for key, val in metrics.items():
    print(key, f'{val.item():.6f}')

observed: 0 suggested: x = 0.435287, y = 0.580127
observed: 1 suggested: x = 0.020918, y = 0.495109
observed: 2 suggested: x = 0.103798, y = 0.471585
observed: 3 suggested: x = 0.094196, y = 0.473273
groun truth: x = 0.127678, y = 0.932751
loss 0.001414
accuracy 0.000000
mae 0.037607


In [195]:
i = np.random.randint(0, len(val_online_dataset))
sample = val_online_dataset[i]
results = model.run(
    problem=sample["problem"],
    n_steps=4
)
outputs_unsqueezed = results["outputs"].unsqueeze(0), sample["x_min"].unsqueeze(0), results["x"].unsqueeze(0)
metrics = model.get_metrics(*outputs_unsqueezed) | model.get_loss(*outputs_unsqueezed)

transform = lambda x: ''.join([f'{x:.6f}' for x in x.tolist()])
for i, (x, y) in enumerate(zip(results["x"], results["y"])):
    print(f'observed: {i} suggested: x = {transform(x)}, y = {y:.6f}')
print(f'groun truth: x = {transform(sample["problem"].info["x_min"])}, y = {sample["problem"].info["y_min"]:.6f}')

for key, val in metrics.items():
    print(key, f'{val.item():.6f}')

observed: 0 suggested: x = 0.435287, y = 0.122911
observed: 1 suggested: x = 0.098243, y = 1.670192
observed: 2 suggested: x = 0.716114, y = 0.921985
observed: 3 suggested: x = 0.731917, y = 1.023368
groun truth: x = 0.457487, y = 0.116980
accuracy 0.000000
mae 0.274430
loss 0.075312


In [175]:
results["outputs"].shape
sample["x_min"].shape
results["x"].shape

torch.Size([4, 1])